In [64]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/root/")

from nnsight import LanguageModel
from buffer import AllActivationBuffer
from datasets import load_dataset
import torch as t

device = "cuda:0" if t.cuda.is_available() else "cpu"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
model = LanguageModel("gpt2", device_map=device)
model.eval()
print("Model loaded")

Model loaded


In [ ]:
dataset = load_dataset(
    'Skylion007/openwebtext', 
    split='train', 
    streaming=True,
    trust_remote_code=True
    )

class CustomData():
    def __init__(self, dataset):
        self.data = iter(dataset)

    def __iter__(self):
        return self

    def __next__(self):
        return next(self.data)['text']

data = CustomData(dataset)

TODO: separate io value for each module. Also figure out something for embed and unembed?

In [88]:
n_layer = model.config.n_layer
submodule_list = [model.transformer.h[layer].mlp for layer in range(n_layer)] + [model.transformer.h[layer].attn for layer in range(n_layer)]
batch_size_tokens = 512

buffer = AllActivationBuffer(
    data=data,
    model=model,
    submodules=submodule_list,
    d_submodule=768, # output dimension of the model component
    n_ctxs=128,  # you can set this higher or lower depending on your available memory
    device="cuda",
    out_batch_size = batch_size_tokens,
    refresh_batch_size = 256,
    io = 'in_and_out'
)  # buffer will yield batches of tensors of dimension = num_submodules * d_submodule


In [85]:
batch = next(buffer)

In [86]:
x = list(batch.values())[0]
x.shape

torch.Size([512, 2, 768])

In [87]:
batch

{GPT2MLP(
   (c_fc): Conv1D()
   (c_proj): Conv1D()
   (act): NewGELUActivation()
   (dropout): Dropout(p=0.1, inplace=False)
 ): tensor([[[ 3.2386,  1.6644,  1.0096,  ...,  0.4195, -2.1544, -0.9841],
          [ 3.2386,  1.6644,  1.0096,  ...,  0.4195, -2.1544, -0.9841]],
 
         [[ 1.5712,  2.0126,  1.6756,  ..., -2.3381, -0.8188,  5.1985],
          [ 1.5712,  2.0126,  1.6756,  ..., -2.3381, -0.8188,  5.1985]],
 
         [[ 0.4695,  1.0103, -0.7286,  ..., -0.6898,  0.1669,  0.4679],
          [ 0.4695,  1.0103, -0.7286,  ..., -0.6898,  0.1669,  0.4679]],
 
         ...,
 
         [[-0.0941, -2.1715, -1.6023,  ...,  1.6938, -0.4226,  1.7260],
          [-0.0941, -2.1715, -1.6023,  ...,  1.6938, -0.4226,  1.7260]],
 
         [[-1.0349, -0.1961,  0.7401,  ..., -1.1925,  0.6921,  0.1126],
          [-1.0349, -0.1961,  0.7401,  ..., -1.1925,  0.6921,  0.1126]],
 
         [[-1.8163, -0.7129,  0.3570,  ...,  1.7247, -1.0194,  2.3416],
          [-1.8163, -0.7129,  0.3570,  ...,  1.7